# Example 04: Get a specific dataset from ILO API

This example shows how to get a specific dataset from ILO's SDMX API (ILOSTAT SDMX Gateway). 

### Load required libraries

In [1]:
import xmltodict
import json
import requests 
import xml.etree.ElementTree as ET 

### Interactive shell

In [2]:
# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Get data from API endpoint and save as xml file on disk

#### Build URL string

In [3]:
baseURL = 'http://www.ilo.org/sdmx/rest/'

countryCode = 'MEX'

indicator = 'EAP_DWAP_SEX_AGE_RT'

# url of API endpoint returning all the dissemination dataflows (i.e. indicators) with data available
url = baseURL + 'data/ILO,DF_YI_'+ countryCode + '_' + indicator + '?format=csv'  # 

url


'http://www.ilo.org/sdmx/rest/data/ILO,DF_YI_MEX_EAP_DWAP_SEX_AGE_RT?format=csv'

In [10]:
import csv
import requests

with requests.Session() as s:
    download = s.get(url)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

['<?xml version="1.0" encoding="ISO-8859-1"?>']
['<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"']
['  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">']
['<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">']
['<head>']
['<title>Object not found!</title>']
['<link rev="made" href="mailto:webtransfers@ilo.org" />']
['<style type="text/css"><!--/*--><![CDATA[/*><!--*/ ']
['    body { color: #000000; background-color: #FFFFFF; }']
['    a:link { color: #0000CC; }']
['    p', ' address {margin-left: 3em;}']
['    span {font-size: smaller;}']
['/*]]>*/--></style>']
['</head>']
[]
['<body>']
['<h1>Object not found!</h1>']
['<p>']
[]
[]
['    The requested URL was not found on this server.']
[]
['  ']
[]
['    If you entered the URL manually please check your']
['    spelling and try again.']
[]
['  ']
[]
['</p>']
['<p>']
['If you think this is a server error', ' please contact']
['the <a href="mailto:webtransfers@ilo.org">webmaster</a>.']
[]
['</p>']
[]
['<h2>E

#### Download csv to disk

In [4]:
# creating HTTP response object from given url 
resp = requests.get(url) 

resp

# saving the into an csv file 
with open('test2.csv', 'wb') as f: 
    f.write(resp.content) 

1032

### Open xml file using `xmltodict.parse`
This will create an object consisting of various nested ordered dictionaries. Every node of the XML tree is an ordered dictionary of key-value pairs. The root of the XML tree is an ordered dictionary itself.

In [ ]:
xmlFile = 'test.xml'

with open(xmlFile, encoding="utf-8") as fd:
    doc = xmltodict.parse(fd.read())


In [ ]:
doc.keys()

### Drill down to level 3, which contains the actual code list

In [ ]:
print('---Root:')
len(doc)
rootKey = list(doc.keys())[0]
rootKey

print('---Level 1:')
level1 = doc[rootKey]
len(level1)
level1Keys = list(level1.keys())
level1Keys

print('---Level 2 / message:DataSet')
level2 = level1['message:DataSet']
len(level2)
level2Keys = list(level2.keys())
level2Keys

print('---Level 3 / generic:Series')
level3 = level2['generic:Series']
len(level3)

level3[2]


### Collect the concept scheme information in a dictionary

In [ ]:
records = []

for r in level3:

    observations = r['generic:Obs']
  

    if isinstance(observations, list):
        for o in observations:
            record = {}

            seriesKey = r['generic:SeriesKey']

            for k in seriesKey['generic:Value']:
                record[k['@concept']] = k['@value']
                
            record['year'] = o['generic:Time']
            record['value'] = o['generic:ObsValue']['@value']
            for a in o['generic:Attributes']['generic:Value']:
                record[a['@concept']] = a['@value']
            
            records.append(record)
    else:
        record = {}

        seriesKey = r['generic:SeriesKey']

        for k in seriesKey['generic:Value']:
            record[k['@concept']] = k['@value']

        record['year'] = observations['generic:Time']
        record['value'] =observations['generic:ObsValue']['@value']
        for a in observations['generic:Attributes']['generic:Value']:
            record[a['@concept']] = a['@value']
            
        records.append(record)
        
records[0:10]


        
    

In [ ]:
records = []

for r in level3:
    record = {}

    seriesKey = r['generic:SeriesKey']
    dataFlow['id'] = df['@id']
    dataFlow['urn'] = df['@urn']
    dataFlow['version'] = df['@version']
    dataFlow['isFinal'] = df['@isFinal']
    
    dataFlow['nameEN'] = None
    dataFlow['nameES'] = None
    dataFlow['nameFR'] = None

    dataFlowName = df['str:Name']

    if isinstance(dataFlowName, list):
        for x in dataFlowName:
            if x['@xml:lang'] == 'en':
                dataFlow['nameEN'] = x['#text']
            if x['@xml:lang'] == 'es':
                dataFlow['nameES'] = x['#text']
            if x['@xml:lang'] == 'fr':
                dataFlow['nameFR'] = x['#text']
    else:
        if dataFlowName['@xml:lang'] == 'en':
            dataFlow['nameEN'] = dataFlowName['#text']
        if dataFlowName['@xml:lang'] == 'es':
            dataFlow['nameES'] = dataFlowName['#text']
        if dataFlowName['@xml:lang'] == 'fr':
            dataFlow['nameFR'] = dataFlowName['#text']
            
    
    
    dataFlow['descriptionEN'] = None
    dataFlow['descriptionES'] = None
    dataFlow['descriptionFR'] = None
            
    if 'str:Description' in df.keys():
        dataFlowDescription = df['str:Description']

        if isinstance(dataFlowDescription, list):
            for x in dataFlowDescription:
                if x['@xml:lang'] == 'en':
                    dataFlow['descriptionEN'] = x['#text']
                if x['@xml:lang'] == 'es':
                    dataFlow['descriptionES'] = x['#text']
                if x['@xml:lang'] == 'fr':
                    dataFlow['descriptionFR'] = x['#text']
        else:
            if dataFlowDescription['@xml:lang'] == 'en':
                dataFlow['descriptionEN'] = dataFlowDescription['#text']
            if dataFlowDescription['@xml:lang'] == 'es':
                dataFlow['descriptionES'] = dataFlowDescription['#text']
            if dataFlowDescription['@xml:lang'] == 'fr':
                dataFlow['descriptionFR'] = dataFlowDescription['#text']
   
    
    keyFamilyRef = df['str:KeyFamilyRef'] 
    
    dataFlow['keyFamilyID'] = keyFamilyRef['str:KeyFamilyID']
    dataFlow['keyFamilyAgencyID'] = keyFamilyRef['str:KeyFamilyAgencyID']
    
    
    dataFlows.append(dataFlow)
    
dataFlows
    